# Flowers Image Classification with TPUs on Cloud ML Engine

This notebook demonstrates how to do image classification from scratch on a flowers dataset using TPUs

In [1]:
import os
PROJECT = 'cloud-training-demos' # REPLACE WITH YOUR PROJECT ID
BUCKET = 'cloud-training-demos-ml' # REPLACE WITH YOUR BUCKET NAME
REGION = 'us-central1' # REPLACE WITH YOUR BUCKET REGION e.g. us-central1

# do not change these
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION

In [2]:
%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


## Convert JPEG images to TensorFlow Records

My dataset consists of JPEG images in Google Cloud Storage. I have two CSV files that are formatted as follows:
   image-name, category

Instead of reading the images from JPEG each time, we'll convert the JPEG data and store it as TF Records.


In [26]:
%bash
gsutil cat gs://cloud-ml-data/img/flower_photos/train_set.csv | head -5 > /tmp/input.csv
cat /tmp/input.csv

gs://cloud-ml-data/img/flower_photos/daisy/754296579_30a9ae018c_n.jpg,daisy
gs://cloud-ml-data/img/flower_photos/dandelion/18089878729_907ed2c7cd_m.jpg,dandelion
gs://cloud-ml-data/img/flower_photos/dandelion/284497199_93a01f48f6.jpg,dandelion
gs://cloud-ml-data/img/flower_photos/dandelion/3554992110_81d8c9b0bd_m.jpg,dandelion
gs://cloud-ml-data/img/flower_photos/daisy/4065883015_4bb6010cb7_n.jpg,daisy


In [54]:
%bash
gsutil cat gs://cloud-ml-data/img/flower_photos/train_set.csv  | sed 's/,/ /g' | awk '{print $2}' | sort | uniq > /tmp/labels.txt
cat /tmp/labels.txt

daisy
dandelion
roses
sunflowers
tulips


## Run preprocessing

First try it out locally -- note that the inputs are all local paths

In [61]:
%bash
export PYTHONPATH=${PYTHONPATH}:${PWD}/imgclass
python -m trainer.preprocess \
       --inputCsv /tmp/input.csv \
       --labelsFile /tmp/labels.txt \
       --projectId $PROJECT \
       --outputDir /tmp/out \
       --outputPrefix train

Read in 5 labels, from daisy to tulips


/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
2018-06-08 07:39:14.890483: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA


In [62]:
!ls -l /tmp/out

total 548
-rw-r--r-- 1 root root 558761 Jun  8 07:39 train-00000-of-00001


Now run it over full training and evaluation datasets.  This will happen in Cloud Dataflow, so go to the GCP console and wait for the jobs to finish.

In [ ]:
%bash
export PYTHONPATH=${PYTHONPATH}:${PWD}/imgclass

# the files have to be called train or validation as these strings are hardcoded into the amoebanet code
python -m trainer.preprocess \
       --inputCsv gs://cloud-ml-data/img/flower_photos/train_set.csv \
       --labelsFile /tmp/labels.txt \
       --projectId $PROJECT \
       --outputDir gs://${BUCKET}/tpu/imgclass \
       --outputPrefix train &

python -m trainer.preprocess \
       --inputCsv gs://cloud-ml-data/img/flower_photos/eval_set.csv \
       --labelsFile /tmp/labels.txt \
       --projectId $PROJECT \
       --outputDir gs://${BUCKET}/tpu/imgclass \
       --outputPrefix validation &

## Train on the Cloud

Now, let's do it on ML Engine. Note the --model parameter

## Deploying and predicting with model

Deploy the model:

In [ ]:
%bash
MODEL_NAME="flowers"
MODEL_VERSION=${MODEL_TYPE}
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/flowers/trained_${MODEL_TYPE}/export/Servo | tail -1)
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ... this will take a few minutes"
gcloud ml-engine versions delete --quiet ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ml-engine models delete ${MODEL_NAME}
#gcloud ml-engine models create ${MODEL_NAME} --regions $REGION
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION}

To predict with the model, let's take one of the example images that is available on Google Cloud Storage <img src="http://storage.googleapis.com/cloud-ml-data/img/flower_photos/sunflowers/1022552002_2b93faf9e7_n.jpg" />

In [48]:
%writefile test.json
{"imageurl": "gs://cloud-ml-data/img/flower_photos/sunflowers/1022552002_2b93faf9e7_n.jpg"}

Overwriting test.json


Send it to the prediction service

In [58]:
%bash
gcloud ml-engine predict --model=flowers --version=${MODEL_TYPE} --json-instances=./test.json

CLASS       CLASSID  PROBABILITIES
sunflowers  3        [0.005260971374809742, 0.0005250766407698393, 0.008500268682837486, 0.9802882671356201, 0.0054253824055194855]


<pre>
# Copyright 2017 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
</pre>